In [ ]:
# check this one!
import numpy as np
import cv2
import imageio
from skimage.measure import label, regionprops
import operator
import time
from tqdm import tqdm_notebook
import math
from PIL import Image

%gui qt

writeVidOut = False
verbose = True
pip = True

In [ ]:
if 'fgbg' not in locals():
    fgbg = cv2.createBackgroundSubtractorKNN()
    
morph_size = 2
shadowValue = 127
learnBG = False
showShadow = True

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(morph_size,morph_size))

reader = imageio.get_reader('fgmasktest.mkv')
# reader = imageio.get_reader('FW501.mp4')
fps = reader.get_meta_data()['fps'] 
print('input video file has a framerate of {} fps'.format(fps))

if writeVidOut:
    writer = imageio.get_writer('test-segments-pip-pts.mp4', fps=fps)
# see https://github.com/imageio/imageio/issues/84
# http://imageio.readthedocs.io/en/latest/installation.html

startTime = 0

width, height = reader.get_meta_data()['size']

ratOnlyImg = fgbg.getBackgroundImage() # really just need empty image of correct size

try:
    for i, im in enumerate(tqdm_notebook(reader, desc='tracking object')):
        if i%30 == 0:
            endTime = time.time()
            procFPS = int(30/(endTime - startTime))
            startTime = time.time()

        #im =  cv2.cvtColor(im, cv2.COLOR_RGB2BGR) # imageio writer takes RGB, so convert it to BGR
        im =  cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

        if learnBG:
            fgmask = fgbg.apply(im)
        else:
            fgmask = fgbg.apply(im, learningRate=0)

#         fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
        fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(8*morph_size,8*morph_size)))
        if verbose:
            bg = fgbg.getBackgroundImage()

        # see https://www.mathworks.com/matlabcentral/answers/68696-how-can-i-extract-the-largest-blob-in-a-binary-image
        label_img = label(fgmask)
        regions = regionprops(label_img)

        region_areas = []

        for props in regions:
            region_areas.append(props.area)

        im =  cv2.cvtColor(im, cv2.COLOR_GRAY2BGR) # for opencv color display

        if len(region_areas) > 0:
            largestBlobIndex, _ = max(enumerate(region_areas), key=operator.itemgetter(1))

            ratBlob = regions[largestBlobIndex]
            
            
            ratOnlyImg[:] = 0
            for px in ratBlob.coords:
                ratOnlyImg[px[0], px[1]] = 255

            y00, x00 = ratBlob.centroid
            
            orientation = ratBlob.orientation
            
            x1 = x00 + math.cos(orientation) * 0.5 * ratBlob.major_axis_length
            y1 = y00 - math.sin(orientation) * 0.5 * ratBlob.major_axis_length
#             x2 = x0 - math.sin(orientation) * 0.5 * ratBlob.minor_axis_length
#             y2 = y0 - math.cos(orientation) * 0.5 * ratBlob.minor_axis_length
#             x3 = x0 + math.sin(orientation) * 0.5 * ratBlob.minor_axis_length
#             y3 = y0 + math.cos(orientation) * 0.5 * ratBlob.minor_axis_length

            x2 = x00 - math.sin(orientation) * 0.55 * ratBlob.minor_axis_length
            y2 = y00 - math.cos(orientation) * 0.55 * ratBlob.minor_axis_length
            x3 = x00 + math.sin(orientation) * 0.55 * ratBlob.minor_axis_length
            y3 = y00 + math.cos(orientation) * 0.55 * ratBlob.minor_axis_length

#             cv2.line(im,(int(x0), int(y0)), (int(x1), int(y1)), color=(0,255,255), thickness=2)

            # SPLIT RAT IN HALF:
            cv2.line(ratOnlyImg,(int(x00), int(y00)), (int(x2), int(y2)), color=(0,0,0), thickness=2)
            cv2.line(ratOnlyImg,(int(x00), int(y00)), (int(x3), int(y3)), color=(0,0,0), thickness=2)
            
        
#             ax.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
#             ax.plot(x0, y0, '.g', markersize=15)

            label_img = label(ratOnlyImg)
            regions = regionprops(label_img)
        
            writeVidOut = False
            verbose = True
            pip = True  
            
            if len(regions) > 1:
                rathalf = regions[0]
                y0, x0 = rathalf.centroid

                orientation = rathalf.orientation

    #                 x1 = x0 + math.cos(orientation) * 0.5 * half.major_axis_length
    #                 y1 = y0 - math.sin(orientation) * 0.5 * half.major_axis_length
                x1 = x0 - math.cos(orientation) * 0.5 * rathalf.major_axis_length
                y1 = y0 + math.sin(orientation) * 0.5 * rathalf.major_axis_length
    #             x2 = x0 - math.cos(orientation) * 0.5 * rathalf.major_axis_length
    #             y2 = y0 + math.sin(orientation) * 0.5 * rathalf.major_axis_length

                cv2.line(im,(int(x0), int(y0)), (int(x1), int(y1)), color=(255,150,150), thickness=2)
                cv2.line(im,(int(x00), int(y00)), (int(x0), int(y0)), color=(200,150,150), thickness=2)
                cv2.circle(im,(int(x0),int(y0)),3,(255,100,255),-11)
                cv2.circle(im,(int(x1),int(y1)),3,(255,255,255),-11)

                cv2.circle(im,(int(x00),int(y00)),10,(255,255,255),-11)
                cv2.circle(im,(int(x00),int(y00)),11,(0,0,255),1) # draw circle
                cv2.ellipse(im, (int(x00),int(y00)), (10,10), 0, 0, 90,(0,0,255),-1 )
                cv2.ellipse(im, (int(x00),int(y00)), (10,10), 0, 180, 270,(0,0,255),-1 )
                cv2.circle(im,(int(x00),int(y00)),1,(0,255,0),1) # draw center

                
                x2 = x0 - math.sin(orientation) * 0.55 * rathalf.minor_axis_length
                y2 = y0 - math.cos(orientation) * 0.55 * rathalf.minor_axis_length
                x3 = x0 + math.sin(orientation) * 0.55 * rathalf.minor_axis_length
                y3 = y0 + math.cos(orientation) * 0.55 * rathalf.minor_axis_length

    #             cv2.line(im,(int(x0), int(y0)), (int(x1), int(y1)), color=(0,255,255), thickness=2)

                # SPLIT RAT IN HALF:
                cv2.line(ratOnlyImg,(int(x0), int(y0)), (int(x2), int(y2)), color=(0,0,0), thickness=2)
                cv2.line(ratOnlyImg,(int(x0), int(y0)), (int(x3), int(y3)), color=(0,0,0), thickness=2)
            
                rathalf = regions[1]
                y0, x0 = rathalf.centroid

                orientation = rathalf.orientation

    #                 x1 = x0 + math.cos(orientation) * 0.5 * half.major_axis_length
    #                 y1 = y0 - math.sin(orientation) * 0.5 * half.major_axis_length
                x1 = x0 + math.cos(orientation) * 0.5 * rathalf.major_axis_length
                y1 = y0 - math.sin(orientation) * 0.5 * rathalf.major_axis_length
    #             x2 = x0 - math.cos(orientation) * 0.5 * rathalf.major_axis_length
    #             y2 = y0 + math.sin(orientation) * 0.5 * rathalf.major_axis_length

                cv2.line(im,(int(x0), int(y0)), (int(x1), int(y1)), color=(150,200,255), thickness=2)
                cv2.line(im,(int(x00), int(y00)), (int(x0), int(y0)), color=(150,170,200), thickness=2)
                cv2.circle(im,(int(x0),int(y0)),3,(255,100,255),-11)
                cv2.circle(im,(int(x1),int(y1)),3,(255,255,255),-11)
                
                x2 = x0 - math.sin(orientation) * 0.55 * rathalf.minor_axis_length
                y2 = y0 - math.cos(orientation) * 0.55 * rathalf.minor_axis_length
                x3 = x0 + math.sin(orientation) * 0.55 * rathalf.minor_axis_length
                y3 = y0 + math.cos(orientation) * 0.55 * rathalf.minor_axis_length

    #             cv2.line(im,(int(x0), int(y0)), (int(x1), int(y1)), color=(0,255,255), thickness=2)

                # SPLIT RAT IN HALF:
                cv2.line(ratOnlyImg,(int(x0), int(y0)), (int(x2), int(y2)), color=(0,0,0), thickness=2)
                cv2.line(ratOnlyImg,(int(x0), int(y0)), (int(x3), int(y3)), color=(0,0,0), thickness=2)
                
        label_img = label(ratOnlyImg)
        regions = regionprops(label_img)
        
        for props in regions:
            x000, y000 = props.centroid
            cv2.circle(im,(int(y000),int(x000)),3,(0,255,0),-11)
            cv2.circle(ratOnlyImg,(int(y000),int(x000)),3,(0,225,0),-11)
            
            
        
        if pip:
            size = 260, 320
            pipframe = Image.fromarray(ratOnlyImg)
            
            pipframe.thumbnail(size, Image.ANTIALIAS) 
            size = pipframe.size
            im[height-size[1]:height,0:size[0],0] = np.array(pipframe)
            im[height-size[1]:height,0:size[0],1] = np.array(pipframe)
            im[height-size[1]:height,0:size[0],2] = np.array(pipframe)
            
            pipframe = Image.fromarray(fgmask)
            
            pipframe.thumbnail(size, Image.ANTIALIAS) 
            size = pipframe.size
            im[0:size[1],width-size[0]:width,0] = np.array(pipframe)
            im[0:size[1],width-size[0]:width,1] = np.array(pipframe)
            im[0:size[1],width-size[0]:width,2] = np.array(pipframe)
        
        
        cv2.putText(im,str(procFPS) + ' fps',(width - 90, 20),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255))
        timestamp = int(i/fps)
        m, s = divmod(timestamp, 60)
        h, m = divmod(m, 60)
        timeStr = "%d:%02d:%02d" % (h, m, s)
        cv2.putText(im,timeStr,(width - 90, 40),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255))

        
        
        if verbose:
            cv2.imshow('fgmask',fgmask)
            cv2.imshow('bg',bg)
            cv2.imshow('segments', ratOnlyImg)

        cv2.imshow('im',im)

        im =  cv2.cvtColor(im, cv2.COLOR_BGR2RGB) # imageio writer takes RGB
        #im =  cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)

        if writeVidOut:
            writer.append_data(im)

        k = cv2.waitKey(1) & 0xff
    #    if k!= 255:
    #        print(k)
        if k == 32: # 'space'
            if learnBG:
                learnBG = False
                print('background learning OFF')
            else:
                learnBG = True
                print('background learning ON')
        if k == 115: # 's'
            if showShadow:
                showShadow = False
                shadowValue = 0
                print('shadows OFF')
            else:
                showShadow = True
                shadowValue = 127
                print('shadows ON')
            #fgbg.setDetectShadows(showShadow)
            fgbg.setShadowValue(shadowValue)

        if k == 171 or k == 43: # '+'
            if morph_size < 20:
                morph_size +=1
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(morph_size,morph_size))
        if k == 173 or k == 45: # '-'
            if morph_size > 2:
                morph_size -=1
                kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(morph_size,morph_size))
        if k == 27:
            break

    if writeVidOut:
        writer.close()
    print('exited gracefully')
except RuntimeError:
    print('something went wrong')
    
cv2.destroyAllWindows()